# UPS Plus (EP-0136) バッテリーモニター

52Pi UPS Plus から I2C 経由でバッテリー情報を取得します。

- **0x17**: UPS MCU (バッテリー電圧, 容量, 充電ポート検出)
- **0x40**: INA219 供給電力モニター (RPi 消費電力)
- **0x45**: INA219 バッテリーモニター (充放電電流)

In [ ]:
from ina219 import INA219
import smbus2

I2C_BUS = 1
MCU_ADDR = 0x17


def read_mcu_16(bus, reg_lo):
    """UPS MCU から 16bit LE 値を読み出す (公式と同じ read_byte_data 方式)"""
    lo = bus.read_byte_data(MCU_ADDR, reg_lo)
    hi = bus.read_byte_data(MCU_ADDR, reg_lo + 1)
    return (hi << 8) | lo


def read_all():
    """全センサーを読み取って返す。バスは読み取り後に閉じる。"""
    bus = smbus2.SMBus(I2C_BUS)
    try:
        mcu = {
            "mcu_v": read_mcu_16(bus, 1),
            "rpi_v": read_mcu_16(bus, 3),
            "batt_v": read_mcu_16(bus, 5),
            "usb_c_v": read_mcu_16(bus, 7),
            "micro_usb_v": read_mcu_16(bus, 9),
            "temp": read_mcu_16(bus, 11),
            "cap": read_mcu_16(bus, 19),
        }
    finally:
        bus.close()

    ina_supply = INA219(0.00725, busnum=I2C_BUS, address=0x40)
    ina_supply.configure()
    supply = {"v": ina_supply.voltage(), "i": ina_supply.current(), "p": ina_supply.power()}
    ina_supply._i2c.close()

    ina_batt = INA219(0.005, busnum=I2C_BUS, address=0x45)
    ina_batt.configure()
    batt = {"v": ina_batt.voltage(), "i": ina_batt.current(), "p": ina_batt.power()}
    ina_batt._i2c.close()

    return mcu, supply, batt


print("UPS Plus ヘルパー定義完了")

In [ ]:
# === 全ステータス取得 ===
mcu, supply, batt = read_all()

print("=== Supply (RPi) ===")
print(f"  電圧:   {supply['v']:.2f} V")
print(f"  電流:   {supply['i']:.1f} mA")
print(f"  電力:   {supply['p']:.1f} mW")

print("\n=== Battery (INA219) ===")
print(f"  電圧:   {batt['v']:.2f} V")
print(f"  電流:   {batt['i']:.1f} mA ({'充電' if batt['i'] > 0 else '放電'})")
print(f"  電力:   {batt['p']:.1f} mW")

print("\n=== MCU (0x17) ===")
print(f"  MCU 電圧:         {mcu['mcu_v']} mV")
print(f"  RPi 出力電圧:     {mcu['rpi_v']} mV")
print(f"  バッテリー電圧:   {mcu['batt_v']} mV")
print(f"  USB-C 電圧:       {mcu['usb_c_v']} mV")
print(f"  MicroUSB 電圧:    {mcu['micro_usb_v']} mV")
print(f"  バッテリー温度:   {mcu['temp']} ℃")
print(f"  残量:             {mcu['cap']} %")

if mcu["usb_c_v"] > 4000:
    print("\n  → USB-C から充電中")
elif mcu["micro_usb_v"] > 4000:
    print("\n  → MicroUSB から充電中")
else:
    print("\n  → 非充電")

In [ ]:
# === バッテリー電圧の推移を記録 ===
import time
import matplotlib.pyplot as plt

DURATION = 60   # 秒
INTERVAL = 2    # 秒

timestamps = []
voltages = []
currents = []

print(f"{DURATION}秒間モニタリング開始 ({INTERVAL}秒間隔)")
start = time.time()
while time.time() - start < DURATION:
    t = time.time() - start
    _, _, batt = read_all()
    timestamps.append(t)
    voltages.append(batt["v"])
    currents.append(batt["i"])
    print(f"  {t:5.1f}s  V={batt['v']:.2f}V  I={batt['i']:.0f}mA", end="\r")
    time.sleep(INTERVAL)

print(f"\n{len(timestamps)} サンプル取得完了")

In [ ]:
# === グラフ表示 ===
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

ax1.plot(timestamps, voltages, "b-")
ax1.set_ylabel("Battery Voltage (V)")
ax1.grid(True)

ax2.plot(timestamps, currents, "r-")
ax2.set_ylabel("Battery Current (mA)")
ax2.set_xlabel("Time (s)")
ax2.axhline(y=0, color="gray", linestyle="--", linewidth=0.5)
ax2.grid(True)

fig.suptitle("UPS Plus Battery Monitor")
plt.tight_layout()
plt.show()